# Skin Lesion Segmentation Training

In [1]:
from datetime import datetime
from ex_utils import run_all_experiments, run_eval_from_csv
from models import list_models

# ============================================
# 설정 (Configuration)
# ============================================

class Config:
    # ========== Eval-Only 모드 ==========
    EVAL_ONLY = False  # True: 기존 모델 추론만, False: 학습
    EVAL_CSV_PATH = 'results/260219_174828/results.csv'  # 기존 결과 CSV 경로
    
    # ========== 베타 테스트 모드 ==========
    BETA_TEST = False  # True: 1 step per epoch (코드 검증용)
    
    # ========== 시각화 모드 ==========
    VISUALIZE_OUTPUT = False  # True: 테스트 출력 시각화 분석 진행

    start_from =(1, 1, 1)
    # 시드 범위 [시작, 끝]
    seed_range = [1, 5]
        
    # 데이터셋 선택
    datasets_list = [
        'PH2',
        'isic17', 
        'isic18', 
    ]
    # 모델 선택
    networks = [
        # 'JeongWonNet_CtxBridge_StdExp',
        'JeongWonNet_CtxBridge_PostStdExp_mod',
        'JeongWonNet_CtxBridge_PostStdExp_mod_GlobalCtx',
        # 'EGEUNet',
        'UCMNet',
        # 'MALUNet',
        # 'CMUNeXt',
        # 'TinyUNet',
        # 'MHorUNet',
        'MAResUNet',
        # 'AMNet',
    ]
    
    # ========== 모델별 Loss 선택 ==========
    # 매핑에 없는 모델은 기존 GT_BceDiceLoss (BCE+Dice) 사용
    # 사용 가능: 'boundary_focal', 'lovasz', 'uncertainty'
    model_loss_map = {
        'JeongWonNet_CtxBridge_PostStdExp_mod_GlobalCtx': 'boundary_focal',
        'JeongWonNet_CtxBridge_PostStdExp_mod': 'boundary_focal',
        'JaengWonNet_CtxBridge_PostStdExp_GlobalCtx': 'boundary_focal',
        'JeongWonNet_CtxBridge_PostStdExp': 'boundary_focal',
        'JeongWonNet_Shuffle': 'boundary_focal',
        'JeongWonNet_CtxBridge_StdExp_Boun2': 'boundary_focal',
        'JeongWonNet_CtxBridge_StdExp_Boun': 'boundary_focal',
        'JeongWonNet_CtxBridge_StdExp': 'boundary_focal',
        'MAResUNet': 'bce',
        'UCMNet': 'bce_dice_squared',
        # 'MALUNet': 'uncertainty',
    }
    
    # 데이터 경로
    data_paths = {
        'isic17': './data/isic2017/',
        'isic18': './data/isic2018/',
        'PH2': './data/PH2/'
    }
    
    # 학습 하이퍼파라미터
    batch_size = 16
    epochs = 100
    num_workers = 4
    gpu_id = '0'
    
    # 이미지 크기
    input_size_h = 256
    input_size_w = 256
    
    # 로깅
    print_interval = 20
    val_interval = 30
    save_interval = 100
    threshold = 0.5
    
    # Optimizer
    opt = 'AdamW'
    lr = 0.001
    betas = (0.9, 0.999)
    eps = 1e-8
    weight_decay = 1e-2
    amsgrad = False
    
    # Scheduler
    sch = 'CosineAnnealingLR'
    T_max = epochs
    eta_min = 1e-6
    last_epoch = -1
    
    # 실험 시간
    experiment_time = datetime.now().strftime('%y%m%d_%H%M%S')
    results_root = f'results/{experiment_time}/'

config = Config()

# 베타 테스트 모드: 출력 폴더만 변경
if config.BETA_TEST:
    config.results_root = f'results/BETA_{config.experiment_time}/'

# 실험 정보 출력
if config.EVAL_ONLY:
    print(f'[EVAL-ONLY MODE]')
    print(f'CSV: {config.EVAL_CSV_PATH}')
else:
    seeds = list(range(config.seed_range[0], config.seed_range[1] + 1))
    total = len(seeds) * len(config.datasets_list) * len(config.networks)
    print(f'Total experiments: {total}')
    print(f'Models: {config.networks}')
    print(f'Datasets: {config.datasets_list}')
    print(f'Seeds: {seeds}')
    
if config.VISUALIZE_OUTPUT:
    print(f'[VISUALIZE MODE] 테스트 출력 시각화 활성화')

if hasattr(config, 'model_loss_map') and config.model_loss_map:
    print(f'Custom losses: {config.model_loss_map}')
    
print(f'Available models: {list_models()}')

Total experiments: 60
Models: ['JeongWonNet_CtxBridge_PostStdExp_mod', 'JeongWonNet_CtxBridge_PostStdExp_mod_GlobalCtx', 'UCMNet', 'MAResUNet']
Datasets: ['PH2', 'isic17', 'isic18']
Seeds: [1, 2, 3, 4, 5]
Custom losses: {'JeongWonNet_CtxBridge_PostStdExp_mod_GlobalCtx': 'boundary_focal', 'JeongWonNet_CtxBridge_PostStdExp_mod': 'boundary_focal', 'JaengWonNet_CtxBridge_PostStdExp_GlobalCtx': 'boundary_focal', 'JeongWonNet_CtxBridge_PostStdExp': 'boundary_focal', 'JeongWonNet_Shuffle': 'boundary_focal', 'JeongWonNet_CtxBridge_StdExp_Boun2': 'boundary_focal', 'JeongWonNet_CtxBridge_StdExp_Boun': 'boundary_focal', 'JeongWonNet_CtxBridge_StdExp': 'boundary_focal', 'MAResUNet': 'bce', 'UCMNet': 'bce_dice_squared'}
Available models: ['AMNet', 'AMNet_Nearest', 'AttU_Net', 'CMUNeXt', 'DWBlock', 'EGEUNet', 'JaengWonNet_CtxBridge_PostStdExp_GlobalCtx', 'JeongWonNet', 'JeongWonNet77', 'JeongWonNet77_Rep', 'JeongWonNet77_Rep256', 'JeongWonNet77_Rep256Basis8', 'JeongWonNet77_Rep256Basis8B2', 'JeongWo

In [ ]:
if config.EVAL_ONLY:
    # 기존 모델로 추론만 수행
    df_results = run_eval_from_csv(config, config.EVAL_CSV_PATH)
else:
    # 일반 학습 
    df_results, df_summary = run_all_experiments(config, start_from = config.start_from)

print('\n[Results]')
if df_results is not None:
    print(df_results.to_string(index=False))

Resuming from: seed[1]=1, dataset[1]=PH2, model[1]=JeongWonNet_CtxBridge_PostStdExp_mod
Total: 60 experiments | Seeds: [1, 2, 3, 4, 5] | Datasets: ['PH2', 'isic17', 'isic18'] | Models: ['JeongWonNet_CtxBridge_PostStdExp_mod', 'JeongWonNet_CtxBridge_PostStdExp_mod_GlobalCtx', 'UCMNet', 'MAResUNet']
[1/60] JeongWonNet_CtxBridge_PostStdExp_mod_PH2_seed_1
1EP(02/22 23:09:07): T_Loss: 7.544452 V_Loss: 5.216446 IoU: 0.4057 *Best V_Loss: 5.216446
